In [ ]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
music = pd.read_csv("/content/spotify_millsongdata.csv")

In [ ]:
music.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
music.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [ ]:
music.shape

(57650, 4)

In [ ]:
music.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [ ]:
music = music.sample(10000).drop('link', axis = 1).reset_index(drop = True)

In [ ]:
music.head()

,artist,song,text
0,Indigo Girls,Free In You,A hard knock \r\nA cold clock \r\nTicking of...
1,Tom T. Hall,Raking Up Leaves,All summer long you smelled like lotion and we...
2,Bob Seger,Sometimes,Sometimes I move in chains \r\nBut I never fe...
3,Yolanda Adams,The Prayer,I pray you'll be our eyes \r\nAnd watch us wh...
4,Tragically Hip,Boots Or Hearts,Well I think that there's a problem here \r\n...


In [ ]:
music['text'][0]

"A hard knock  \r\nA cold clock  \r\nTicking off my time  \r\nA long look  \r\nBut no luck  \r\nCouldn't seem to find  \r\nOr unwind  \r\nInto peace of mind  \r\nThough I was trying  \r\n  \r\nA quick glance  \r\nA big chance  \r\nMy heart beat like a drum  \r\nI saw you  \r\nAnd I knew  \r\nChances just don't come  \r\nRound again  \r\nNot like this  \r\nFirst a laugh  \r\nThen the kiss  \r\n  \r\nAnd I'm free in you  \r\nI've got no worries on my mind  \r\nI know what to do  \r\nThat's to treat you right  \r\nAnd love you kind  \r\nThank you ever on my mind  \r\nLove is just like breathing  \r\nWhen it's true  \r\nAnd I'm free in you  \r\n  \r\nThe lost time  \r\nThe self crime  \r\nMy big mistakes  \r\nThe clear voice  \r\nOf bad choice  \r\nSounding like an ache  \r\nIn my day  \r\nNot too bad  \r\nBut too real  \r\nTo go away  \r\n  \r\nBut now I'm free in you  \r\nI got no worries on my mind  \r\nI know what to do  \r\nThat's to treat you right  \r\nAnd love you kind  \r\nThank y

In [ ]:
music.shape

(10000, 3)

In [ ]:
music['text']=music['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n',' ', regex = True)

In [ ]:
music.head(100)

,artist,song,text
0,Indigo Girls,Free In You,a hard knock \r a cold clock \r ticking off ...
1,Tom T. Hall,Raking Up Leaves,all summer long you smelled like lotion and we...
2,Bob Seger,Sometimes,sometimes i move in chains \r but i never fee...
3,Yolanda Adams,The Prayer,i pray you'll be our eyes \r and watch us whe...
4,Tragically Hip,Boots Or Hearts,well i think that there's a problem here \r h...
...,...,...,...
95,Diana Ross,In Your Arms,"hold me, touch me, make me your woman \r i wi..."
96,The Temptations,"Hello, Young Lovers","hello, young lovers whoever you are. \r i hop..."
97,Marillion,Between You And Me,today! i saw music in the sky \r i drove towa...
98,Jimmy Buffett,Everybody's On The Run,first time i ran \r was to the end of the blo...


In [ ]:
music.tail(5)

,artist,song,text
9995,Oasis,She Is Love,oh when the sunshine beckons to ya \r and you...
9996,Randy Travis,Old Time Christmas,the ol log is burnin' \r yellow and red \r s...
9997,Bing Crosby,Oh What A Beautiful Mornin,"there's a bright golden haze on the meadow, \..."
9998,Phish,Boogie On Reggae Woman - Phish,i like to see you boogie \r right across the ...
9999,Tim McGraw,Take Me Away From Here,my body burns \r like there's a desert deep i...


In [ ]:
stemmer = PorterStemmer()

In [ ]:
def token(text):
  token = nltk.word_tokenize(text)
  a =[stemmer.stem(w) for w in token ]
  return " ".join(a)

In [ ]:
token("YA HILU, beauti")

'ya hilu , beauti'

In [ ]:
music['text'].apply(lambda x: token(x))

0       a hard knock a cold clock tick off my time a l...
1       all summer long you smell like lotion and we m...
2       sometim i move in chain but i never feel the p...
3       i pray you 'll be our eye and watch us where w...
4       well i think that there 's a problem here her ...
                              ...                        
9995    oh when the sunshin beckon to ya and your wing...
9996    the ol log is burnin ' yellow and red small ey...
9997    there 's a bright golden haze on the meadow , ...
9998    i like to see you boogi right across the floor...
9999    my bodi burn like there 's a desert deep in me...
Name: text, Length: 10000, dtype: object

In [ ]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
matrix = tfid.fit_transform(music['text'])

In [ ]:
lister =cosine_similarity(matrix)

In [ ]:
lister[0]

array([1.        , 0.04299365, 0.02371512, ..., 0.0389023 , 0.03836592,
       0.08074821])

In [ ]:
music[music['song'] == 'Old Time Christmas'].index[0]

9996

Recommanding FUnction

In [ ]:
def recommender(song_name):
  index = music[music['song'] == song_name].index[0]
  dist = sorted(list(enumerate(lister[index])), reverse = True , key = lambda x:x[1])
  song = []
  for song_id in dist[1:5]:
    song.append(music.iloc[song_id[0]].song)
  return song

In [ ]:
recommender("Thank You Lord")

["Rasta Man's Prayer", 'Thank You', 'Doubly Good To You', 'Oh My Lord']

In [ ]:
pickle.dump(lister , open("similarity", "wb"))

In [ ]:
pickle.dump(music, open("music","wb"))